# PROBLEM 4 EM for coin flips

## Read data

In [1]:
import requests


url = 'https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/coin_flips_outcome.txt'

response = requests.get(url)


In [2]:
data = response.text

with open('coinflips.txt', 'w') as file:
    file.write(data)


In [7]:
import pandas as pd

d = {}

for i in range(20):
    d['col' + str(i)] = []

with open("coinflips.txt", "r") as file:
    for line in file:
        values = line.split(' ')
        for i in range(20):
            d['col' + str(i)].append(int(values[i]))

df = pd.DataFrame(d)


In [10]:
df.shape

(50, 20)

In [11]:
df.to_csv('coinflips.csv', index=False)

## Probability computation

In [1]:
import pandas as pd

df = pd.read_csv('coinflips.csv')
df.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19
0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,1,0,1
1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,1
2,1,1,0,1,1,1,0,0,1,1,0,0,0,1,0,1,1,0,1,0
3,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [2]:
arr = df.to_numpy()

In [3]:
arr.shape

(50, 20)

In [4]:
import numpy as np
import random
from scipy.stats import multivariate_normal

def initialize_parameters(arr, K):
    d = arr.shape[1]
    indices = np.random.choice(len(arr), size=K, replace=False)
    mus = np.mean(arr[indices], axis=1)

    return mus, np.array([1/K for k in range(K)])

K=3
mus, wi = initialize_parameters(arr, K)

In [20]:
mus

array([0.5 , 0.95, 0.55])

In [5]:
from scipy.stats import binom

def e_step(arr, wi, mus, K):
    probs = []
    sucessful_occ = np.sum(arr, axis=1)
    n = arr.shape[1]

    for k in range(K):
        probs.append(binom.pmf(sucessful_occ, n, mus[k]))
    probs = np.array(probs)
    probs = probs * wi.reshape(K,1)
    probs_sum = np.sum(probs, axis=0)
    resp = probs / probs_sum
    return resp

resp = e_step(arr, wi, mus, K)

In [7]:
resp.shape

(3, 50)

In [32]:
def m_step(arr, pi_vals, K):
    mus = []
    n = arr.shape[0]
    d = arr.shape[1]
    arr = np.mean(arr, axis=1)

    for k in range(K):
        kth_resp = pi_vals[k]
        kth_resp_sum = np.sum(kth_resp)
        mu = np.sum(arr.reshape(n,1) * kth_resp.reshape(n,1)) / kth_resp_sum
        mus.append(mu)
    wi = np.sum(pi_vals, axis=1) / n
    return np.array(mus), wi

mus, wi =  m_step(arr, resp, K)

In [37]:
def log_likelihood(arr, mus, wi, K):
    obj_val = 0
    n = arr.shape[0]
    sucessful_occ = np.sum(arr, axis=1)
    
    for i in range(K):
        k_sum = 0.0
        for j in range(K):
            probs = binom.pmf(sucessful_occ, n, mus[i])
            k_sum += np.sum(wi[i] * probs)
        obj_val += np.log(k_sum + 1e-10)
    return obj_val

In [38]:
log_likelihood(arr, mus, wi, K)

-26.278798831274457

In [45]:
def converge(arr, K, max_itr=100, tol=1e-6):
    mus, wi = initialize_parameters(arr, K)
    
    for itr in range(max_itr):
        resp = e_step(arr, wi, mus, K)
        mus, wi = m_step(arr, resp, K)
        ll = log_likelihood(arr, mus, wi, K)
        print("Error", ll)

    return mus, wi



In [46]:
mus, wi = converge(arr, K, max_itr=100, tol=1e-6)

Error -33.59418943068139
Error -31.704954310650333
Error -30.010264291027298
Error -28.83098856208511
Error -28.1195977592874
Error -27.698569006365975
Error -27.452997829880076
Error -27.310982675476865
Error -27.228121321812676
Error -27.179196807523667
Error -27.150047074405542
Error -27.132575909601897
Error -27.122065591055453
Error -27.1157284617202
Error -27.111902274828914
Error -27.10959020229888
Error -27.108192368141044
Error -27.107347006935615
Error -27.106835667688134
Error -27.106526335983766
Error -27.106339194925987
Error -27.106225972755446
Error -27.10615747054072
Error -27.106116024385695
Error -27.106090947834154
Error -27.106075775452524
Error -27.106066595485053
Error -27.10606104118424
Error -27.106057680573734
Error -27.1060556472465
Error -27.106054416987437
Error -27.106053672622366
Error -27.10605322224613
Error -27.106052949747085
Error -27.10605278487218
Error -27.10605268511502
Error -27.106052624757183
Error -27.10605258823782
Error -27.106052566141855
E

In [47]:
mus

array([0.61003783, 0.93172853, 0.23691867])

In [48]:
wi

array([0.51462834, 0.17855768, 0.30681398])